In [3]:
import pandas as pd
import dash 
import numpy as np
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go

In [68]:
#Hopefully this page still works
df=pd.read_html('https://www.usinflationcalculator.com/inflation/historical-inflation-rates/')[0]
df=df.melt(id_vars='Year', var_name='Month', value_name='YoY')
#Clean
df.dropna(inplace=True)
# #That's not a month
df=df[df['Month']!='Ave']
# #Make date usable
df['Date']=df['Year'].astype('str')+df['Month']
df['Date']=pd.to_datetime(df['Date'], format='%Y%b')
#Final fix
df=df[~df['YoY'].astype('str').str.contains('Avail.')][['Date', 'YoY']]
df['YoY']=[float(x) for x in df['YoY']]
df.sort_values('Date', inplace=True, ignore_index=True)
df.set_index('Date', inplace=True)
df

,YoY
Date,
1914-01-01,2.0
1914-02-01,1.0
1914-03-01,1.0
1914-04-01,0.0
1914-05-01,2.1
...,...
2023-03-01,5.0
2023-04-01,4.9
2023-05-01,4.0


In [32]:
def calculate_yoy(row, years, df_ref):
    """
    Calculates the compounded inflation change over the specified number of years.

    Parameters:
    - row: The DataFrame row for which the calculation is being done.
    - years: Number of years over which to calculate the YoY change.
    - df_ref: Reference DataFrame with the historical data.

    Returns:
    - Compounded inflation change for the specified number of years.
    """
    
    # Extract the starting date from the row's index
    start_date = row.name

    # Fetch the YoY values for the specified number of years
    yoy_values = [df_ref.loc[start_date - pd.DateOffset(years=i), 'YoY'] 
                  for i in range(years) if start_date - pd.DateOffset(years=i) in df_ref.index]
    
    # If any year's data is missing, return None
    if len(yoy_values) != years:
        return None

    # Calculate the compounded YoY change
    total = np.prod([(yoy / 100 + 1) for yoy in yoy_values])

    # Convert the total change to percentage and round off
    return round((total - 1) * 100, 1)


In [105]:
# Create the Dash app
app = dash.Dash(__name__)

# Create an input box
input_box = dcc.Input(id='input-box', type='number', placeholder='Years of Inflation')

# Create a container for the plot
plot = dcc.Graph(id='plot')

# Input fields for start and end years
start_year_input = dcc.Input(id='start-year-input', type='number', placeholder='Start Year', value=min(unique_years))
end_year_input = dcc.Input(id='end-year-input', type='number', placeholder='End Year', value=max(unique_years))

# Create a storage component
storage = dcc.Store(id='storage', data=[])

# Arrange the components in the app layout
app.layout = html.Div([input_box, start_year_input, end_year_input, dcc.Loading(
        id="loading-plot",
        type="circle",  # choose from "default", "circle", or "cube"
        children=[plot]  # wrap the component that should show the spinner when loading
    ), html.Div(id='values-container'),storage], style={'backgroundColor': 'white'})

# Update the plot when a value is input
@app.callback(
    [Output('plot', 'figure'),
     Output('storage', 'data')],
    [Input('input-box', 'value'),
     Input('start-year-input', 'value'),
     Input('end-year-input', 'value')],
    [State('storage', 'data')]
)
def update_plot(input_value, start_year, end_year, data):
    # Process input_value to ensure it's valid
    if input_value is None:
        input_value = 1  # set default value if input is None
    years = int(input_value)
    # If the year is already in the data, remove it and skip the rest of the callback
    if years in data:
        data.remove(years)
        fig = go.Figure()
        for year in data:
            fig.add_trace(go.Scatter(x=df.index, y=df['{} Year'.format(year)], name='{} Year'.format(year)))
        return fig, data

    # Apply the function to each row in the DataFrame
    df['{} Year'.format(years)] = df.apply(lambda row: calculate_yoy(row, years, df), axis=1)
    # Use the entire DataFrame, since there's no dropdown anymore
    df_filtered = df.copy()

    # Apply date filter
    df_filtered = df[(df.index.year >= start_year) & (df.index.year <= end_year)]

    data.append(years)
    # Create the new figure and return it
    fig = go.Figure()
    # Filter the dataframe to include only the relevant 'Inflation Change' columns
    columns_to_include = ['{} Year'.format(year) for year in data]
    df_plot = df_filtered[columns_to_include]
    # Use plotly.express to create the line chart
    fig = px.line(df_plot, title='Inflation Change Over Years', labels={'value': 'Inflation Change', 'index': 'Date', 'variable': 'Years'})
    fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    xaxis_gridcolor='gray',
    yaxis_gridcolor='gray')
    fig.update_yaxes(zerolinecolor='black')
    return fig, data 

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
    